In [30]:
import json
from pathlib import Path
import tensorflow as tf
import src.data.build_dataset as build_dataset
import src.data.input_pipeline as input_pipeline
import importlib
import src.models.transformer.metrics as metrics

importlib.reload(metrics)

acc =tf.metrics.CategoricalAccuracy()
pred = tf.random.uniform(shape=(1,2,25))
true = tf.random.uniform(shape=(1,2,25))
pred_pos = tf.constant([[[0]]])
true_pos = tf.constant([1])
categories = tf.constant([[1,2]])
metrics.fitb_acc(pred,true,pred_pos,true_pos, categories, acc)
print(acc.result())


tf.Tensor([0], shape=(1,), dtype=int32)
tf.Tensor([1], shape=(1,), dtype=int32)
tf.Tensor([[0 1]], shape=(1, 2), dtype=int32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [ ]:
file = [r"D:\David\Škola\outfit-generation\data\processed\tfrecords\fitb-features.tfrecord"]
importlib.reload(input_pipeline)
dataset = input_pipeline.get_fitb_dataset(file, True).take(1)

for x in dataset:
    inputs, input_categories, targets, target_categories, target_position = x
    
    print(inputs)
    
    print(input_categories)